# Ejercicio Práctico: Análisis de Recompra en una Campaña de Marketing
**Instituto Tecnológico Beltrán – Modelizado de Minería de Datos**

Este notebook realiza un análisis completo sobre la efectividad de una campaña de marketing,
buscando identificar los factores que influyen en la recompra de clientes mediante un modelo
de **árbol de decisión**.


In [ ]:
# === IMPORTACIONES ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import warnings
warnings.filterwarnings('ignore')
plt.ion()


## Carga de Datos
**Nota:** Cambia la ruta del archivo por la ubicación donde esté guardado el Excel en tu PC.


In [ ]:
# Cargar el archivo Excel
# 👉 Reemplaza la ruta por donde se aloje tu archivo
df = pd.read_excel("C:\\Users\\estudiante\\Desktop\\TareasPYTHON\\2025\\MMD\\Mini_Proyecto_Clientes_Promociones.xlsx")

# Exploración básica
display(df.head())
df.info()
df.describe()


## Análisis Exploratorio Inicial

In [ ]:
print("Distribución de valores categóricos:")
print(df['Genero'].value_counts())
print(df['Recibio_Promo'].value_counts())
print(df['Recompra'].value_counts())

# Efectividad de promociones
total_clientes = len(df)
con_promo = df[df['Recibio_Promo'] == 'Sí']
sin_promo = df[df['Recibio_Promo'] == 'No']
recompra_con_promo = len(con_promo[con_promo['Recompra'] == 'Sí'])
recompra_sin_promo = len(sin_promo[sin_promo['Recompra'] == 'Sí'])

print(f"Tasa de recompra CON promoción: {recompra_con_promo/len(con_promo)*100:.1f}%")
print(f"Tasa de recompra SIN promoción: {recompra_sin_promo/len(sin_promo)*100:.1f}%")


## Modelado Predictivo

In [ ]:
df_procesado = df.copy()
df_procesado['Genero'] = df_procesado['Genero'].map({'F': 0, 'M': 1})
df_procesado['Recibio_Promo'] = df_procesado['Recibio_Promo'].map({'Sí': 1, 'No': 0})
df_procesado['Recompra'] = df_procesado['Recompra'].map({'Sí': 1, 'No': 0})

X = df_procesado.drop(['Cliente_ID', 'Recompra'], axis=1)
y = df_procesado['Recompra']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo = DecisionTreeClassifier(max_depth=4, random_state=42)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

feature_importance = pd.DataFrame({
    'Variable': X.columns,
    'Importancia': modelo.feature_importances_
}).sort_values('Importancia', ascending=False)

print("Matriz de Confusión:")
print(cm)
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print(f"Precisión del modelo: {accuracy*100:.2f}%")
feature_importance


## Dashboard Estático con Tkinter

In [ ]:
def mostrar_dashboard(df, feature_importance, accuracy,
                      recompra_con_promo, con_promo,
                      recompra_sin_promo, sin_promo):
    root = tk.Tk()
    root.title("Dashboard de Recompra - Campaña de Marketing")
    root.geometry("1200x700")
    root.configure(bg="#f5f5f5")

    tk.Label(root, text="Dashboard Ejecutivo - Recompra de Clientes",
             font=("Segoe UI", 18, "bold"), bg="#f5f5f5").pack(pady=10)

    frame_graficos = ttk.Frame(root)
    frame_graficos.pack(pady=10, padx=10)

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    fig.tight_layout(pad=3)

    # Gráfico 1: Efectividad
    tasas = pd.DataFrame({
        'Categoría': ['Con Promoción', 'Sin Promoción'],
        'Tasa de Recompra (%)': [
            recompra_con_promo / len(con_promo) * 100,
            recompra_sin_promo / len(sin_promo) * 100
        ]
    })
    sns.barplot(x='Categoría', y='Tasa de Recompra (%)',
                data=tasas, palette=['#4ecdc4', '#ff6b6b'], ax=axes[0])
    axes[0].set_title("Efectividad de Promociones")
    for i, v in enumerate(tasas['Tasa de Recompra (%)']):
        axes[0].text(i, v + 1, f"{v:.1f}%", ha='center', fontsize=9)

    # Gráfico 2: Variables
    top_features = feature_importance.head(5)
    sns.barplot(x='Importancia', y='Variable',
                data=top_features, ax=axes[1], palette="viridis")
    axes[1].set_title("Variables Más Importantes")

    # Gráfico 3: Distribución
    recompra_counts = df['Recompra'].value_counts()
    axes[2].pie(recompra_counts, labels=['No Recompra', 'Recompra'],
                autopct='%1.1f%%', colors=['#ff6b6b', '#4ecdc4'], startangle=90)
    axes[2].set_title("Distribución General")

    canvas = FigureCanvasTkAgg(fig, master=frame_graficos)
    canvas.draw()
    canvas.get_tk_widget().pack()

    texto = f"""
Precisión del modelo: {accuracy*100:.2f}%
Variables Clave:
• {feature_importance.iloc[0]['Variable']}
• {feature_importance.iloc[1]['Variable']}
• {feature_importance.iloc[2]['Variable']}
Conclusión: El modelo predice la recompra con buena precisión.
"""
    tk.Label(root, text=texto, font=("Consolas", 11),
             bg="#fff8e1", justify="left", anchor="w",
             relief="solid", padx=10, pady=10).pack(fill="x", padx=20, pady=10)

    tk.Button(root, text="Cerrar Dashboard", command=root.destroy,
              bg="#ff6b6b", fg="white", font=("Segoe UI", 11, "bold"),
              relief="flat", padx=15, pady=5).pack(pady=10)

    root.mainloop()

plt.close('all')
mostrar_dashboard(df, feature_importance, accuracy,
                  recompra_con_promo, con_promo,
                  recompra_sin_promo, sin_promo)
